In [ ]:
# USE GPU

In [ ]:
!pip install tensorflow_hub
!pip install opencv-contrib-python --upgrade
!pip install ruptures

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/AIART/src')

In [4]:
from Parameters import params, directory_params
from StyleGAN2 import StyleGAN2
from Setup import SetUp, get_noise
from Audioreactive import AudioReactive
from Train import make_dataset
from Train import DisplayCallBack

/content/drive/MyDrive/AIART/src/Setup.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
import tensorflow as tf
import librosa
import matplotlib.pyplot as plt

In [6]:
img_size = 2**params.log2res
ds_directory = os.path.join(directory_params.base_directory, directory_params.dataset_folder)
audiofile = os.path.join(directory_params.music_directory, directory_params.audio_name)
melody_audiofile = os.path.join(directory_params.music_directory, directory_params.melody_name)
drums_audiofile = os.path.join(directory_params.music_directory, directory_params.drums_name)
bass_audiofile = os.path.join(directory_params.music_directory, directory_params.bass_name)
vox_audiofile = os.path.join(directory_params.music_directory, directory_params.vox_name)

# Compile and Train

In [ ]:
tf.keras.backend.clear_session()
stylegan2 = StyleGAN2(log2_resolution=params.log2res, batch_size=None, latent_dim=512, min_num_features=16, max_num_features=params.max_features, name='StyleGAN2')
stylegan2.compile(steps_per_epoch=params.steps, run_eagerly=False)
stylegan2.build((2, 4, 4, params.max_features))
stylegan2.summary()

Model: "StyleGAN2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 discriminator (Discriminato  multiple                 0 (unused)
 r)                                                              
                                                                 
 generator (Generator)       multiple                  17285736  
                                                                 
 mapping (Mapping)           multiple                  2101248   
                                                                 
 adaptive_augmenter (Adaptiv  multiple                 0 (unused)
 eAugmenter)                                                     
                                                                 
Total params: 39,230,191
Trainable params: 39,230,179
Non-trainable params: 12
_________________________________________________________________


In [ ]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(ds_directory, label_mode=None, image_size=(img_size, img_size), batch_size=params.BATCH_SIZE)
ds_train = ds_train.map(lambda x: x)
checkpoint = os.path.join(directory_params.training_checkpoint_directory, '4_17_22/Final_Test/', 'ckpt')
ckpt_cb = tf.keras.callbacks.ModelCheckpoint(checkpoint, save_weights_only=True, save_freq=2000, verbose=0)
train_dl = make_dataset(2**params.log2res, ds_train, 2)

Found 20885 files belonging to 1 classes.


In [ ]:
params.steps = 2001

In [ ]:
stylegan2.fit(train_dl, epochs=1, steps_per_epoch=params.steps, callbacks=[ckpt_cb, DisplayCallBack()])

In [ ]:
stylegan2.load_weights(os.path.join(directory_params.training_checkpoint_directory, directory_params.ckpt_folder_name, directory_params.ckpt_name))

In [ ]:
im = stylegan2.generate_images(1)
plt.imshow(im[0])

# AudioReactive

In [7]:
params.duration = 451
# directory_params.output_dir = ''

In [8]:
audio, sr_audio = librosa.load(audiofile, offset=0, duration=params.duration)
melody, sr_melody = librosa.load(melody_audiofile, offset=0, duration=params.duration)
drums, sr_drums = librosa.load(drums_audiofile, offset=0, duration=params.duration)
bass, sr_bass = librosa.load(bass_audiofile, offset=0, duration=params.duration)
vox, sr_vox = librosa.load(vox_audiofile, offset=0, duration=params.duration)

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


In [9]:
num_frames = int(round(params.duration*params.fps))

In [10]:
setup = SetUp(audio=audio, duration=params.duration, melody=melody, drums=drums, bass=bass, sr=sr_audio, num_frames=num_frames)
setup.setup(True, 36, 2)

/usr/local/lib/python3.7/dist-packages/librosa/filters.py:239: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  "Empty filters detected in mel frequency basis. "


In [12]:
noise = get_noise(num_frames, params.log2res)

  0%|          | 0/7 [00:00<?, ?it/s]

In [13]:
MakeAudioReeactive = AudioReactive(max_features=params.max_features, 
                                   noise=noise, 
                                   audio=audio, 
                                   audio_file=audiofile, 
                                   melody=melody,
                                   drums=drums, 
                                   bass=bass, 
                                   sr=sr_audio, 
                                   num_frames=num_frames, 
                                   setup=setup,
                                   train_ckpt_base_dir=directory_params.training_checkpoint_directory,
                                   ckpt_folder_name=directory_params.ckpt_folder_name,
                                   ckpt_name=directory_params.ckpt_name)

In [15]:
MakeAudioReeactive.generate_video(out_loc=directory_params.output_dir, img_loc=os.path.join(directory_params.output_dir, 'imgs'), super_res=False)

Getting Preperations...
Generating Images...


  0%|          | 0/10824 [00:00<?, ?it/s]

  0%|          | 0/10824 [00:00<?, ?it/s]

Adding Audio to Video...
